# CIML Predictions

In this notebook we train and evaluate CIML experiments using the functions `gather_results` and `tf_trainer functions` of the [ciml project](https://github.com/mtreinish/ciml). 
<br>Then we save the predictions of the experiments for a deeper analysis of the metrics of the trained models (see [CIML Metric Report](https://nbviewer.jupyter.org/github/kwulffert/ciml_experiments/blob/master/Metrics%20report.ipynb)).

In [17]:
from ciml import gather_results
from ciml import tf_trainer
import numpy as np
import pandas as pd
import collections
import tensorflow as tf
from tensorflow.python.training import adagrad
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import logging
tf.get_logger().setLevel(logging.ERROR)

First we define the data path, dataset and experiment to gather the right input dataset and the configuration for the experiment.

In [18]:
data_path = '/Users/kw/ciml_data/cimlodsceu2019seed'

#Dataset and experiment combination for multiple classification
#dataset = 'usr_1m-1min-node_provider'
#experiment = 'dnn-3x100-500epochs-bs128'

#Dataset and experiment combination for binary classification
dataset = 'usr_1m-1min-status'
experiment = 'dnn-5x100-500epochs-bs128'

In [19]:
experiment_data = gather_results.load_experiment(
        experiment, data_path=data_path)
dataset_data = gather_results.load_model_config(
        dataset, data_path=data_path)

The dataset and experiment_data are dictionaries with the following structure:

In [20]:
dataset_data.keys()

dict_keys(['build_name', 'sample_interval', 'features_regex', 'class_label', 'aggregation_functions', 'training_set', 'dev_set', 'test_set', 'normalized_length', 'labels', 'num_columns', 'num_features', 'normalization_params'])

In [21]:
experiment_data

{'estimator': 'tf.estimator.DNNClassifier',
 'params': {},
 'hyper_params': {'steps': 9500,
  'batch_size': '128',
  'epochs': '500',
  'hidden_units': [100, 100, 100, 100, 100],
  'optimizer': 'Adagrad',
  'learning_rate': 0.05}}

We now set up the experiment and configure the estimator.

In [22]:
estimator = experiment_data['estimator']
hyper_params = experiment_data['hyper_params']
params = experiment_data['params']
steps = int(hyper_params['steps'])
num_epochs = int(hyper_params['epochs'])
batch_size = int(hyper_params['batch_size'])
optimizer = hyper_params['optimizer']
learning_rate = float(hyper_params['learning_rate'])
class_label = dataset_data['class_label']
labels = gather_results.load_dataset(dataset, 'labels', data_path=data_path)['labels']
training_data = gather_results.load_dataset(dataset, 'training', data_path=data_path)
test_data = gather_results.load_dataset(dataset, 'test', data_path=data_path)

#label_vocabulary = None
if class_label == 'node_provider':
    label_vocabulary = set(['rax', 'ovh', 'packethost-us-west-1',
                            'vexxhost', 'limestone-regionone',
                            'inap-mtl01', 'fortnebula-regionone'])
elif class_label == 'node_provider_all':
    label_vocabulary = set(['rax-iad', 'ovh-bhs1', 'packethost-us-west-1',
                            'rax-dfw', 'vexxhost-ca-ymq-1', 'ovh-gra1',
                            'limestone-regionone', 'inap-mtl01', 'rax-ord',
                            'vexxhost-sjc1', 'fortnebula-regionone'])
else:
    label_vocabulary = None

model_dir = gather_results.get_model_folder(dataset, experiment, data_path=data_path)


In [23]:
estimator = tf_trainer.get_estimator(
        estimator, hyper_params, params, labels, model_dir,
        optimizer=adagrad.AdagradOptimizer(learning_rate=0.05),
        label_vocabulary=label_vocabulary, gpu=False)

We train the model.

In [24]:
input_fn=tf_trainer.get_input_fn(shuffle=True,
                    batch_size=batch_size, num_epochs=num_epochs,
                    labels=labels, **training_data)

In [25]:
training_result = tf_trainer.get_training_method(estimator)(
                    input_fn=tf_trainer.get_input_fn(shuffle=True,
                    batch_size=batch_size, num_epochs=num_epochs,
                    labels=labels, **training_data), steps=steps)

We evaluate the trained model with the testset.

In [26]:
eval_data = gather_results.load_dataset(dataset, 'test', data_path=data_path)

In [27]:
eval_size = len(eval_data['example_ids'])
eval_data.keys()

dict_keys(['examples', 'example_ids', 'classes'])

We analyse the predictions of our trained model.
<br>Info logging is enabled to monitor the status of the training.

In [28]:
prediction = estimator.predict(input_fn=tf_trainer.get_input_fn(
                                batch_size=eval_size, num_epochs=1,
                                labels=labels, **eval_data))

In [29]:
predictions = [x for x in prediction]

In [30]:
p_classes = [x["class_ids"][0] for x in predictions]
actual_classes = eval_data["classes"]
classes = zip(p_classes, actual_classes)
counter = collections.Counter(classes)
sorted(counter.values(), reverse=True)[:15]

[744, 37, 7, 6]

In [31]:
counter

Counter({(0, 0): 744, (1, 1): 37, (0, 1): 7, (1, 0): 6})

Let's save the predictions of the trained model in a zipped json file.

In [32]:
serializable_pred = []
for pred in predictions:
    _classes = pred['classes']
    _all_classes = pred['all_classes']
    pred['classes'] = [x.decode("utf-8") for x in _classes]
    pred['all_classes'] = [x.decode("utf-8") for x in _all_classes]
    serializable_pred.append(pred)

prediction_name = "prediction_" + dataset
pred_data = zip(eval_data['example_ids'], serializable_pred,
                eval_data['classes'])
gather_results.save_data_json(
    dataset, [x for x in pred_data],
    prediction_name, sub_folder=experiment, data_path=data_path)